<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    <font color='red'>Mini Jupyter tutorial<br><br>To run each cell, click the cell and press <kbd>Run</kbd> from the menu bar. This will run any Python code or display any text within the selected cell before highlighting the next cell down. There are two types of cell: A <i>text cell</i> of type <kbd>Markdown</kbd> or <kbd>Heading</kbd> and a <i>code cell</i> of type <kbd>Code</kbd> identifiable with the <span style="font-family: courier; color:black; background-color:white;">In[ ]:</span> to the left of the cell</i>. The type of cell is also identifiable from the dropdown menu in the above menu bar to the right of <kbd>Run</kbd>. Any visual results produced by the code (text/figures) are displayed directly below that cell. Press <kbd>Run</kbd> again until you reach the end of the notebook or alternatively click <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart and Run All</kbd>. Should the Jupyter notebook crash for any reason, restart the Jupyter Kernel by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart</kbd>, and start again from the top.
        
</div>

# VariantDB parser

<p style=\"text-align: justify\">
<br>
    This workflow parses and annotates input variant CSV files
</p>

In [1]:
import pandas as pd
import numpy as np
from gnomad_db.database import gnomAD_DB
from natsort import index_natsorted
from number_parser import parse_ordinal
import os
import glob

<div style="background-color:rgb(255, 250, 250); padding:5px;  border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">

<h3 style="text-align: justify">Process all CSV files in the <kbd>input_csv</kbd> folder and annotate them, outputting to the <kbd>output_annotated_csv</kbd> folder</h3>
</div></div>

In [2]:
gnomad_dir='/Users/brettchapman/gnomeAD_DB' # Specify where the SQLite genomad directory is located
internal_db_file='internaldb/internal_database.csv' # Specify the internal database CSV file
goi_file='genes_of_interest/genes.csv' # Specify the genes of interest with 'HGNC Approved Gene Symbol' and 'Ion Reporter Gene Symbol' headers

#Load in the genes of interest
goi = pd.read_csv(goi_file)
    
goi = goi.rename(columns={'HGNC Approved Gene Symbol': 'GeneSymbol'
                              , 'Ion Reporter Gene Symbol': 'Gene'}).reset_index(drop=True)
hgnc_goi_list = list(goi['GeneSymbol'])
internal_goi_list = list(goi['Gene'])

In [3]:
# Read in clinvar database (make sure it has been downloaded with download_clinvar.ipynb)
clinvar_db = pd.read_csv('variant_summary.txt.gz', compression='gzip', sep='\t', low_memory=False)

#Filter Clinvar database for only variants in GRCh38
clinvar_db = clinvar_db[clinvar_db.Assembly == 'GRCh38'].reset_index(drop=True)

# Add in Ref and Alt for merging with input CSV
clinvar_db['Ref'] = clinvar_db['ReferenceAlleleVCF']
clinvar_db['Alt'] = clinvar_db['AlternateAlleleVCF']

#Filter Clinvar database for only genes of interest
clinvar_db = clinvar_db[clinvar_db.GeneSymbol.isin(hgnc_goi_list)]

# Retain only clinvar entries of interest
clinvar_db = clinvar_db[['GeneSymbol', 'Chromosome', 'Start', 'Stop', 'Ref', 'Alt', 'ClinicalSignificance', 'ReviewStatus', 'NumberSubmitters', 'VariationID']]

# Load in internal database
internal_db = pd.read_csv(internal_db_file)
internal_db.dropna(how='all', axis=1, inplace=True)

#dropping check of genome coordinates start and end with internal db
internal_db['Chromosome'] = internal_db['Genomic Coordinates'].apply(lambda x: str(x).split(":")[0])
internal_db = internal_db.drop(columns=['Genomic Coordinates'])

In [4]:
reviewStatus_panel = ['Practice guideline'
                     , 'reviewed by expert panel'
                     , 'criteria provided, multiple submitters, no conflicts']

clinical_pathogenic_panel = ['Pathogenic'
                             , 'Pathogenic, low penetrance'
                             , 'Pathogenic/Likely pathogenic'
                             , 'Likely pathogenic'
                             , 'Likely pathogenic, low penetrance'
                             , 'Established risk allele'
                             , 'Likely risk allele']

clinical_benign_panel = ['Benign'
                         , 'Likely benign'
                         , 'Benign/Likely benign'
                         , 'Uncertain significance'
                         , 'Uncertain risk allele']

In [5]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "input_csv/*.csv"))

In [6]:
def custom_sort(col: pd.Series) -> pd.Series:
    if col.name == "Chromosome":
        return col.apply(parse_ordinal)
    return col

In [7]:
for csv_file in csv_files:
    # read in the input CSV file
    input_csv = pd.read_csv(csv_file, skiprows=[0,1])
    
    output_csv = csv_file.split('/')[-1].split('.csv')[0] + '_annotated.csv'
    
    #Split the Genomic coodinates into Chromosome, start, stop, and assembly columns
    input_csv['Chromosome'] = input_csv['Genomic Coordinates'].apply(lambda x: str(x).split(":")[0])
    input_csv['Start'] = input_csv['Genomic Coordinates'].apply(lambda x: str(x).split(":")[1].split("-")[0])
    input_csv['Stop'] = input_csv['Genomic Coordinates'].apply(lambda x: str(x).split(":")[1].split("-")[1].split(' ')[0])
    input_csv['Start'] = input_csv['Start'].astype(np.int64)
    input_csv['Stop'] = input_csv['Stop'].astype(np.int64)
    input_csv['Assembly'] = input_csv['Genomic Coordinates'].apply(lambda x: str(x).split(":")[1].split("-")[1].split(' ')[1])
    
    # Ensure variants from input CSV only in GRCH38 for cross comparison with clinvar, gnomeAD and internal database
    input_csv = input_csv[input_csv.Assembly == "(GRCH38)"].reset_index(drop=True)
    
    #Filter input CSV for only genes of interest
    input_csv = input_csv[input_csv.Gene.isin(internal_goi_list)]
    
    # Add in the HGNC gene symbol to the input CSV
    input_csv = input_csv.merge(goi, left_on='Gene', right_on='Gene').reset_index(drop=True)
    
    #Merge the input CSV with Clinvar with keys: GeneSymbol, Chromosome, Start, Stop, Ref and Alt
    clinvar_match = input_csv.merge(clinvar_db, left_on=['GeneSymbol', 'Chromosome', 'Start', 'Stop', 'Ref', 'Alt']
                                    , right_on=['GeneSymbol', 'Chromosome', 'Start', 'Stop', 'Ref', 'Alt']).reset_index(drop=True)
    
    # Parse the gnomad SQLite database
    db = gnomAD_DB(gnomad_dir, genome="Grch38")
    
    #Remove some ipykernel Future warnings about concatenating empty dataframes.
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    chromosome_intervals = input_csv[['Chromosome', 'Start', 'Stop']]
    
    df_gnomad = pd.DataFrame()

    for index, row in chromosome_intervals.iterrows():
        chromosome = row['Chromosome']
        start = row['Start']
        stop = row['Stop']
 
        if df_gnomad.empty:
            df_gnomad = db.get_info_for_interval(chrom=chromosome, interval_start=start, interval_end=stop, query="*")
            df_gnomad = df_gnomad[['chrom', 'pos', 'ref', 'alt', 'AF', 'AF_popmax']].reset_index(drop=True)
        else:
            tmp = db.get_info_for_interval(chrom=chromosome, interval_start=start, interval_end=stop, query="*")
            tmp = tmp[['chrom', 'pos', 'ref', 'alt', 'AF', 'AF_popmax']].reset_index(drop=True)        
            if df_gnomad.empty:
                df_gnomad = db.get_info_for_interval(chrom=chromosome, interval_start=start, interval_end=stop, query="*")
                df_gnomad = df_gnomad[['chrom', 'pos', 'ref', 'alt', 'AF', 'AF_popmax']].reset_index(drop=True)            
            else:
                tmp = db.get_info_for_interval(chrom=chromosome, interval_start=start, interval_end=stop, query="*")
                tmp = tmp[['chrom', 'pos', 'ref', 'alt', 'AF', 'AF_popmax']].reset_index(drop=True)
                if not tmp.empty:
                    df_gnomad = pd.concat([df_gnomad, tmp], axis=0).reset_index(drop=True)
  
    df_gnomad = df_gnomad.rename(columns={'chrom': 'Chromosome', 'pos': 'Start', 'ref': 'Ref', 'alt': 'Alt'}).reset_index(drop=True)

    # Merge the parsed gnomad database with the input CSV
    gnomad_match = input_csv.merge(df_gnomad, left_on=['Chromosome', 'Start', 'Ref', 'Alt']
                                   , right_on=['Chromosome', 'Start', 'Ref', 'Alt']).reset_index(drop=True)
    
    # Merge the parsed internal database with the input CSV
    #internal_match = input_csv.merge(internal_db, left_on=['Chromosome', 'Gene', 'Start', 'Variant', 'Transcript']
    #                                 , right_on=['Chromosome', 'Gene', 'Start', 'Variant', 'Transcript']).reset_index(drop=True)
    internal_match = input_csv.merge(internal_db, left_on=['Chromosome', 'Gene', 'Variant', 'Transcript']
                                     , right_on=['Chromosome', 'Gene', 'Variant', 'Transcript']).reset_index(drop=True)
    
    # Merge matches from clinvar, gnomad and internal database, in that order with outer join
    clinvar_gnomad_match = pd.merge(clinvar_match, gnomad_match, how='outer').drop_duplicates()
    internal_clinvar_gnomad_match = pd.merge(clinvar_gnomad_match, internal_match, how='outer').drop_duplicates()
    
    # Merge the annotated entries back into the input CSV add in non matches, with outer join, and then sort
    annotated_csv = pd.merge(internal_clinvar_gnomad_match, input_csv, how='outer').drop_duplicates()
    annotated_csv = annotated_csv.sort_values(by=['Chromosome', 'Gene', 'Start'], key=custom_sort)
    
    # Populate custom classification columns and reason column
    clinvar_conditions = [((annotated_csv.NumberSubmitters >= 3)
                    & (annotated_csv.ReviewStatus.isin(reviewStatus_panel))
                    & (annotated_csv.ClinicalSignificance.isin(clinical_pathogenic_panel))),
                    ((annotated_csv.NumberSubmitters >= 3) 
                    & (annotated_csv.ReviewStatus.isin(reviewStatus_panel))
                    & (annotated_csv.ClinicalSignificance.isin(clinical_benign_panel)))]

    annotated_csv['InternalClassification'] = annotated_csv['InternalClassification'].fillna('0')
    
    clinvar_choices = ['Suspected pathogenic', 'Suspected VUS/benign']
    
    annotated_csv['ClinVarCustomClassification'] = np.select(clinvar_conditions, clinvar_choices)
    
    gnomad_conditions = [(annotated_csv.AF >= 0.01) | (annotated_csv.AF_popmax >= 0.01)]
    
    gnomad_choices = ['Suspected VUS/benign']
    
    annotated_csv['gnomADCustomClassification'] = np.select(gnomad_conditions, gnomad_choices)

    variant_conditions = [(annotated_csv.Variant.astype(str).str.contains(r'\+1{1}[a-zA-Z]|\+2{1}[a-zA-Z]|\-1{1}[a-zA-Z]|\-2{1}[a-zA-Z]'))
                     | (annotated_csv['Protein Change'].astype(str).str.contains(r'Ter|fs'))
                     | (~annotated_csv['Copy Number'].isnull()),
                     (annotated_csv['Prediction Evidence(PVS1)'].astype(str).str.contains(r'intron|UTR'))
                     | (annotated_csv['Protein Change'].str.contains('='))]

    variant_choices = ['Suspected pathogenic', 'Suspected VUS/benign']

    annotated_csv['VariantEffect'] = np.select(variant_conditions, variant_choices)
    
    overall_classify_conditions = [(annotated_csv['InternalClassification'] != '0'),
                               (annotated_csv['InternalClassification'] == '0')
                               & (annotated_csv['ClinVarCustomClassification'] != '0'),
                               (annotated_csv['InternalClassification'] == '0')
                               & (annotated_csv['ClinVarCustomClassification'] == '0')
                               & (annotated_csv['gnomADCustomClassification'] != '0'),
                               (annotated_csv['InternalClassification'] == '0')
                               & (annotated_csv['ClinVarCustomClassification'] == '0')
                               & (annotated_csv['gnomADCustomClassification'] == '0')
                               & (annotated_csv['VariantEffect'] != '0'),
                               (annotated_csv['InternalClassification'] == '0')
                               & (annotated_csv['ClinVarCustomClassification'] == '0')
                               & (annotated_csv['gnomADCustomClassification'] == '0')
                               & (annotated_csv['VariantEffect'] == '0')]

    overall_classify_choices = [
                        list(annotated_csv.InternalClassification),
                        list(annotated_csv.ClinVarCustomClassification),
                        list(annotated_csv.gnomADCustomClassification),
                        list(annotated_csv.VariantEffect),
                        'Assess further']
    
    annotated_csv['ClassificationOverall'] = np.select(overall_classify_conditions, overall_classify_choices)
    
    reason_conditions = overall_classify_conditions[:4]
    
    reason_choices = ['Internal Database', 'ClinVar', 'gnomAD', 'VariantEffect']
    
    annotated_csv['Reason'] = np.select(reason_conditions, reason_choices)
    
    annotated_csv['InternalClassification'].replace(['0', np.nan], '', inplace=True)
    annotated_csv['ClinVarCustomClassification'].replace(['0', np.nan], '', inplace=True)
    annotated_csv['gnomADCustomClassification'].replace(['0', np.nan], '', inplace=True)
    annotated_csv['VariantEffect'].replace(['0', np.nan], '', inplace=True)
    annotated_csv['ClassificationOverall'].replace(['0', np.nan], '', inplace=True)
    annotated_csv['Reason'].replace(['0', np.nan], '', inplace=True)
    
    # Write out the annotated CSV file
    annotated_csv.to_csv('output_annotated_csv/'+ output_csv, index=False)